In [ ]:
from __future__ import print_function
import numpy as np
import sys
def print_DDT(table):
    for row in range(len(table)):
        for col in range(len(table[row])):
            print(table[row][col],end='');
            if col == len(table[row])-1:
                print("\n");
def print_Diff(table):
    for row in range(len(table)):
        for col in range(len(table[row])):
            print(table[row][col],end='');
            if col == len(table[row])-1:
                print("");
def print_Diff_sage(table):
    for row in range(len(table)):
        print("[",end='');
        for col in range(len(table[row])):
            if col == len(table[row])-1:
                print(table[row][col],end=' ');
                print("],",end='');
            else:
                print(table[row][col],end=' , ');
   
#s_box = ((12,6,9,0,1,10,2,11,3,8,5,13,4,14,7,15),); 
#s_box = ((8,5, 2,3, 6,15, 12, 9, 4,11,13, 14, 10, 0, 1,7),);       
s_box = ((0xc,0xa, 0xd,0x3, 0xe,0xb, 0xf, 0x7, 0x8, 0x9, 0x1, 0x5, 0x0, 0x2, 0x4, 0x6),);
#s_box = ((15,14, 13,12, 4,5, 6, 7, 8,9,10, 11, 4, 5, 6,7),);                  
           

#DDT_SIZE = 256
DDT_SIZE = (len(s_box),len(s_box[0]))
#print(DDT_SIZE)
DDT = np.zeros( (DDT_SIZE[1],DDT_SIZE[1]) )
DDT = DDT.astype(int)
sbox_val = []

for p2 in range(DDT_SIZE[1]):
	#col = p2 >> 4
    row = p2 >> 4
	#row = p2 & 15
    col = p2 & 15
    #print(row,col);
    sbox_val.append(s_box[row][col]);

for p1 in range(DDT_SIZE[1]):
	for p2 in range(DDT_SIZE[1]):
		XOR_IN = np.bitwise_xor(p1,p2);
		XOR_OUT = np.bitwise_xor(sbox_val[p1],sbox_val[p2]);
		#print(XOR_OUT)
		DDT[XOR_IN][XOR_OUT] += 1

#print_DDT(DDT);
#DDT[0][0] = 0;
diff_arr = []
impossible_diff_arr=[]
for row in range(len(DDT)):
        row_bin = bin(row)[2:].zfill(4);
        row_arr = [int(i) for i in row_bin];
        for col in range(len(DDT[row])):
            col_bin = bin(col)[2:].zfill(4);
            col_arr = [int(i) for i in col_bin];
            if(DDT[row][col]!=0):
                #DDT_bin = bin(DDT[row][col])[2:].zfill(3);
                if (DDT[row][col] == 16):
                    DDT_bin = '00';
                elif (DDT[row][col] == 4):
                    DDT_bin = '01';
                elif (DDT[row][col] == 2):
                    DDT_bin = '10';
                #elif (DDT[row][col] == 2):
                #    DDT_bin = '100';
                int_DDT_bin = int(DDT_bin,2);
                DDT_arr = [int(i) for i in DDT_bin];
                diff_arr += [row_arr+col_arr+DDT_arr[::-1]];
                for k in range(0,4):
                    #if (k!=DDT[row][col]):
                    if (k!=int_DDT_bin):
                        im_bin = bin(k)[2:].zfill(2);
                        im_arr = [int(i) for i in im_bin];
                        impossible_diff_arr += [row_arr+col_arr+im_arr[::-1]];
            else:
                for k in range(0,4):
                        im_bin = bin(k)[2:].zfill(2);
                        im_arr = [int(i) for i in im_bin];
                        impossible_diff_arr += [row_arr+col_arr+im_arr[::-1]];
#print_Diff(diff_arr);
#print_Diff(impossible_diff_arr)
#print_Diff_sage(diff_arr);
#myPoints = [diff_arr];
poly_test = Polyhedron( vertices = diff_arr)
#i=0
#for v in poly_test.inequality_generator ():
#	print(str(i)+ " " + str(v))
#	i = i +1;
ineq_list = poly_test.inequalities_list();
#print(ineq_list[0:2])
#ineq_list_rotated = [];
#for inq in ineq_list:
#    ineq_list_rotated += [inq[1:]+[inq[0]]]
#print(ineq_list_rotated[0:2])
##ineq_redunction = [1,52,53,131,236,242,408,440,455,481,493,581,586,609,666,711,723,864,926]
#ineq_redunction = [0,52,53,131,236,242,312,440,455,493,507,581,586,609,666,711,723,864,926]
#print(len(ineq_redunction))
#for i in ineq_redunction:
#    print(ineq_list_rotated[i])
#sys.exit()


print(len(ineq_list));
print(len(impossible_diff_arr))
#impossible_diff_arr = impossible_diff_arr[0:50];
#max_count=1;
#while (max_count!=0):
ineq_fail_index = [];
ineq_fail_count = [0]*len(ineq_list);
lp_string=[]
#print("Minimize");
#for i in range(0,len(ineq_list)):
#    print("z"+str(i)+" + ",end='');
#print("\n\n");
#print("Subject To");
lp_string += ["Minimize"];
ineq_str = "";
for i in range(0,len(ineq_list)):
    ineq_str += "z"+str(i)+" + ";
lp_string += [ineq_str[0:-3]];
lp_string += ["\n"];
lp_string += ["Subject To"];
for i in range(0,len(impossible_diff_arr)):
    imp_diff_arr = np.array([1] + impossible_diff_arr[i]);
    #print(,end='');
    ineq_str_i = "";
    for j in range(0,len(ineq_list)):
        ineq_list_arr = np.array(ineq_list[j]);
        if(sum(imp_diff_arr*ineq_list_arr) < 0):
            ineq_fail_index += [(i,j)];
            #print("z"+str(j)+" + ",end='');
            ineq_str_i += "z"+str(j)+" + ";
    #print(" >= 1")
    ineq_str_i = ineq_str_i[0:-3] + " >= 1"
    lp_string += [ineq_str_i];
#print("\n");
#print("Binary");
#for i in range(0,len(ineq_list)):
#    print("z"+str(i));
lp_string += ["\n"];
lp_string += ["Binary"];
for i in range(0,len(ineq_list)):
    lp_string += ["z"+str(i)];
print(len(lp_string));
f1 = open("MinimizeInequality_step_2.lp","w");
for a in lp_string:
    #print(lp_string[i]);
    f1.write(a+"\n");
f1.close();
ineq_list_rotated = [];
for inq in ineq_list:
    ineq_list_rotated += [inq[1:]+[inq[0]]]
#print(ineq_list_rotated[0:2])
#ineq_redunction = [1,52,53,131,236,242,408,440,455,481,493,581,586,609,666,711,723,864,926]
#ineq_redunction = [0,52,53,131,236,242,312,440,455,493,507,581,586,609,666,711,723,864,926]
ineq_redunction = [0, 8, 136, 317, 357, 367, 532, 556, 584, 753, 803, 828, 929, 979, 1025, 1071, 1125, 1186, 1194, 1222]
print(len(ineq_redunction))
for i in ineq_redunction:
    print(ineq_list_rotated[i])




#   max_count = max(ineq_fail_count);
#   print(max_count);
#   if (max_count!=0):
#       #print(ineq_fail_count)
#      max_index_arr = np.where(np.array(ineq_fail_count) == max_count)[0][0];
#       print(max_index_arr);
#       #ineq_list.pop(max_index_arr);
#       ineq_list[max_index_arr] = [0]*9;
#       for i in range(len(ineq_fail_index)-1,-1,-1):
#              if (ineq_fail_index[i][0]==max_index_arr):
#                    #impossible_diff_arr.pop(ineq_fail_index[i][1]);
#                    impossible_diff_arr[ineq_fail_index[i][1]]= [0]*8;

#print(len(ineq_list));
#print(ineq_list);
#print(len(impossible_diff_arr));
#print(impossible_diff_arr);
#a= [(i,c) for element in ineq_fail_count if(element[2]==0)]